# Primitive KML/Z Loader 

## General Info 

This is based off of `fastkml` 

## Motivation 

Existing libraries are pythonic -- they load for use in an object-oriented sense. Instead, I needed something that could load these into simple Numpy/Pandas objects and interact with the data in a rational form. Thus, I'm creating this, to hopefully be able to read through KML and eventually KMZ files and create a comprehensive representation of the data stored. It will, however, by necessity, lose some description statistics. 

## Goals 

### KML Parsing 



In [1]:
from fastkml import kml
import shapely 
import pandas as pd 
import numpy as np 
import lxml 
import json 
import matplotlib.pyplot as plt 
import scipy as sp
from scipy.spatial.distance import pdist, squareform
from shapely.ops import cascaded_union

#garbage managmeent 
import gc 

In [2]:
global counter
counter = 0 
def get_id_from_description(description_string, mark_config = "OBJECTID"): 
    global counter
    if description_string is not None: 
        string = description_string
        string = ' '.join(string.replace("\n", "").split())
        string = (string[string.find("<tr>"):])
        string = "<root>" + string + "</root>"
        tree = lxml.etree.XML(string)
        for row in tree.getchildren(): 
            f = row.getchildren()[0]
            if f.text == mark_config: 
                return int(row.getchildren()[1].text)
    counter += 1
    return counter 

def get_coords(kml_object, resolution=1, mark_config="Beat"): 
    while (isinstance(list(kml_object.features())[0], kml.Document) \
           or isinstance(list(kml_object.features())[0], kml.Folder)): 
        kml_object = list(kml_object.features())[0]
    frames = {} 
    for k in kml_object.features(): 
        geom = shapely.geometry.mapping(k.geometry)
        #now convert 
        arr = geom['coordinates']
        if (len(arr) == 1): 
            arr = [arr] 
        paths = []
        for path in arr:
            path = path[0]
            nparr = np.array(path, dtype=float) 
            subpath = []
            for i in range(0, nparr.shape[0], resolution): 
                row = nparr[i, :].tolist() 
                subpath.append({"lng": row[0], "lat": row[1]})
            paths.append(subpath) 
        geom['coordinates'] = paths 
        frames[get_id_from_description(k.description, mark_config)] = geom  
    return frames 

In [3]:
with open("data/gtboundary_layer.kml", 'rt', encoding='utf-8') as myfile: 
    doc = myfile.read() 

f = kml.KML() 
f.from_string(doc) 

frames = get_coords(f, resolution=1, mark_config="FID") 

In [4]:
#write to file 
with open("shapes.json", "w") as f:
    s = json.dumps(frames)
    s = "var shapes = " + s; 
    f.write(s)

now we have to conver this to 500 feet boundary 

first let's calculate what 1 foot is in terms of longitude and latitude 

This doesn't have to be too exact, so some error is fine 

Thus, we'll define our vectors as such: 

POSTIVE X is MORE POSITIVE LONGITUDE 

POSITIVE Y is MORE POSITIVE LATITUDE 

We'll be using a reverse haversine implementation 

The Haversine Formula Says: 
$$ hav \left(\frac d r\right) = hav(\phi_2 - \phi_1) + \cos(\phi_1)\cos(\phi_2) hav(\lambda_2 - \lambda_1)$$
where
$$ 
hav(\theta) = \sin^2 \left( \frac \theta 2 \right) = \frac {1 - cos(\theta) } {2} $$
and $\phi$ is latitude, $\lambda$ is longitude, both in radians 

In [5]:
def hav(theta): 
    return (1 - np.cos(theta)) / 2

def haversine(lon1, lat1, lon2, lat2): 
    #we expect radians as in put 
    return hav(lat2 - lat1) + np.cos(lat1) * np.cos(lat2) * hav(lon2 - lon1) 

def calc_distance(**k): 
    havs = haversine(k["lon1"], k["lat1"], k["lon2"], k["lat2"])
    c =  2 * np.arctan2(np.sqrt(havs), np.sqrt(1 - havs)) 
    invs = 6371e3 * c  
    return invs
def meters_to_feet(x): 
    return 3.28084 * x

Now, we'll find the average longitude and latitude of our dataset. Then, we'll compute the distance between longitude, holding lat constant, and vice versa. The inverse will give us a lon-lat vector for unit distance. 


In [6]:
#find vals 
lats = [] 
lngs = [] 
for k in frames:
    for b in frames[k]['coordinates']: 
        for a in b: 
            lats.append(a['lat'])
            lngs.append(a['lng'])      

#compute means 
mean_lat = np.deg2rad(np.mean(lats))
mean_lon = np.deg2rad(np.mean(lngs))

#and max/min 
min_lat = np.deg2rad(min(lats)) 
max_lat = np.deg2rad(max(lats))

min_lon = np.deg2rad(min(lngs)) 
max_lon = np.deg2rad(max(lngs)) 

#now find the distances 
movement_along_lat = meters_to_feet(calc_distance(lon1 = min_lon, 
                                                 lat1 = mean_lat, 
                                                 lon2 = max_lon, 
                                                 lat2 = mean_lat))
movement_along_lng = meters_to_feet(calc_distance(lon1 = mean_lon, 
                                                 lat1 = min_lat, 
                                                 lon2 = mean_lon, 
                                                 lat2 = max_lat))
LonPer500Ft = np.rad2deg((max_lon - min_lon) / (movement_along_lat / 500))
LatPer500Ft = np.rad2deg((max_lat - min_lat) / (movement_along_lng / 500))

del lats 
del lngs 
gc.collect() 
coordVect =[ LonPer500Ft, LatPer500Ft]

Now, we need to interpolate to fill in any gaps. We'll use a resolution of 250 as half the required space. 

In [7]:
#Now, linear interpolation to increase resolution: 
def get_distance_in_feet(p1, p2): 
    return meters_to_feet(calc_distance(lon1 =  np.deg2rad(p1['lng']), 
                                       lon2 = np.deg2rad(p2['lng']), 
                                       lat1 = np.deg2rad(p1['lat']), 
                                       lat2 = np.deg2rad(p2['lat'])))
def interpolate(p1, p2): 
    return {'lat': (p1['lat'] + p2['lat'])/2, 
            'lng': (p1['lng'] + p2['lng'])/2}
count = 0 
RESOLUTION = 250
for f in frames: 
    for vi, v in enumerate(frames[f]['coordinates']): 
        k = v 
        finishedInterpolating = False
        inter_count = 0 
        while not finishedInterpolating:
            finishedInterpolating = True 
            interpolated = [] 
            for i in range(0, len(k) - 1): 
                interp = None
                if get_distance_in_feet(k[i], k[i + 1]) > RESOLUTION: 
                    #average coord 
                    interp = interpolate(k[i], k[i + 1])
                interpolated.append(k[i])
                if interp is not None: 
                    finishedInterpolating = False
                    interpolated.append(interp) 
                interpolated.append(k[i + 1]) 
            k = interpolated 
            inter_count += 1
        frames[f]['coordinates'][vi] = k 
        count += 1
        print(count, "/", 34) 
del interpolated 
del k 
del v 
gc.collect() 

1 / 34
2 / 34
3 / 34
4 / 34
5 / 34
6 / 34
7 / 34
8 / 34
9 / 34
10 / 34
11 / 34
12 / 34
13 / 34
14 / 34
15 / 34
16 / 34
17 / 34
18 / 34
19 / 34
20 / 34
21 / 34
22 / 34
23 / 34
24 / 34
25 / 34
26 / 34
27 / 34
28 / 34
29 / 34
30 / 34
31 / 34
32 / 34
33 / 34
34 / 34


0

In [8]:
#Some functions to use later 
def zip_coordinates(coords): 
    xs = list(map(lambda x: x['lng'], coords))
    ys = list(map(lambda y: y['lat'], coords))
    return (xs, ys) 

def plot_dict_coors_on_ax(coords, ax, shape=False): 
    xs, ys = zip_coordinates(coords)
    if not shape: 
        ax.scatter(xs, ys) 
    else: 
        ax.plot(xs, ys)
    return ax 


In [9]:
plt.close('all')
import gc
gc.collect()

0

In [10]:
#Let's go through and regenerate polygons for everything again 
#Ellipse Template 
circle = np.array([i for i in np.linspace(0, 2 * np.pi, 50)]).T 
circle = circle.reshape(circle.shape[0], 1)
circle = np.hstack((np.cos(circle), np.sin(circle))) 
circle = np.multiply(coordVect, circle) 
final_shapes = []
count = 0 
for k in frames: 
    for j in frames[k]['coordinates']: 
        coordinates = zip_coordinates(j) 
        #zip em up 
        zip_coor = list(zip(*coordinates)) 
        #And now create our shape 
        ogShape = shapely.geometry.polygon.Polygon(zip_coor)
        #And create our circles 
        circles = [ogShape] 
        for point in zip_coor: 
            #generate circle 
            circles.append(shapely.geometry.polygon.Polygon(np.add(point, circle))) 
        #Create a Union 
        union_shape = cascaded_union(circles)
        
        #Again, extract the lat-lon coordinates from this. 
        final_shapes.append(union_shape) 
        count += 1
        print(count) 

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34


In [11]:
final_shape = cascaded_union(final_shapes)
#convert to our JSON version 
points_array = [] 
for k in zip(*final_shape.exterior.xy):
    points_array.append({'lng': k[0], 'lat': k[1]})

final_frame = {0: {'coordinates': [points_array]}}
with open("f_shapes.json", "w") as f:
    s = json.dumps(final_frame)
    s = "var o_shapes = " + s; 
    f.write(s)

In [13]:
np.rad2deg(mean_lat), np.rad2deg(mean_lon)

(33.77679009484465, -84.40027944242803)